In [ ]:
import time
import json
import re
import os
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Cấu hình Selenium WebDriver
edge_path = "E:/Crawl Data 2/edgedriver_win64/msedgedriver.exe"
service = Service(edge_path)
browser = webdriver.Edge(service=service)
wait = WebDriverWait(browser, 3)

# Đăng nhập Facebook
def login_facebook():
    print("Đang đăng nhập Facebook...")
    browser.get("https://www.facebook.com/")
    time.sleep(3)

    username = ""
    password = ""

    browser.find_element(By.ID, "email").send_keys(username)
    browser.find_element(By.ID, "pass").send_keys(password)
    browser.find_element(By.NAME, "login").click()
    time.sleep(20)
    print("Đăng nhập thành công.")

# Hàm cuộn trong container bình luận
def scroll_to_bottom():
    print("Bắt đầu cuộn container bình luận...")
    max_attempts = 5
    attempt = 0

    while attempt < max_attempts:
        try:
            comment_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//div[contains(@class, 'x1n2onr6') or contains(@class, 'x1iorvi4') or contains(@class, 'x78zum5') or contains(@class, 'x1q0g3np') or contains(@class, 'x1y1aw1k') or contains(@class, 'x1sxyh0')]")
            ))
            if not comment_container.is_displayed():
                print("Container bình luận không hiển thị, thử lại...")
                attempt += 1
                time.sleep(0.5)
                continue

            print("Đã tìm thấy container bình luận.")
            last_height = browser.execute_script("return arguments[0].scrollHeight", comment_container)
            scroll_count = 0
            max_scrolls = 20
            while scroll_count < max_scrolls:
                browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comment_container)
                time.sleep(0.5)
                new_height = browser.execute_script("return arguments[0].scrollHeight", comment_container)
                if new_height == last_height:
                    print("Đã cuộn hết container bình luận.")
                    return True
                last_height = new_height
                scroll_count += 1
                print(f"Đang cuộn container, lần {scroll_count}...")
        except Exception as e:
            print(f"Không thể cuộn container (lần thử {attempt + 1}): {str(e)}")
            attempt += 1
            time.sleep(0.5)
            if attempt == max_attempts:
                print("Không tìm thấy container, thử cuộn toàn trang...")
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(0.5)
                return False
    return False

# Hàm đếm số lượng bình luận hiện tại
def count_comments():
    try:
        comment_elements = browser.find_elements(By.XPATH, "//div[contains(@class, 'x16hk5td') or contains(@class, 'x12rz0ws')]")
        return len(comment_elements)
    except Exception:
        return 0

# Hàm nhấp vào nút "Xem thêm"
def click_see_more():
    print("Đang nhấp vào các nút 'Xem thêm'...")
    try:
        see_more_buttons = browser.find_elements(By.XPATH, "//div[@role='button' and contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'xem thêm') and not(contains(., 'bình luận'))]")
        if not see_more_buttons:
            print("Không còn nút 'Xem thêm'.")
            return False
        for button in see_more_buttons:
            try:
                browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                if button.is_displayed() and button.is_enabled():
                    wait.until(EC.element_to_be_clickable(button)).click()
                    time.sleep(0.5)
                    print("Đã nhấp một nút 'Xem thêm'.")
                    return True
                else:
                    print("Nút 'Xem thêm' không hợp lệ, bỏ qua.")
            except Exception:
                print("Không thể nhấp 'Xem thêm', bỏ qua.")
        scroll_to_bottom()
    except Exception:
        print("Không tìm thấy nút 'Xem thêm', bỏ qua.")
    return False

# Hàm nhấp vào nút "XXX đã trả lời Y phản hồi"
def click_see_replies():
    print("Đang nhấp vào các nút 'XXX đã trả lời Y phản hồi'...")
    max_attempts = 5
    click_count = 0
    no_new_comments_count = 0
    last_comment_count = count_comments()
    print(f"Số bình luận ban đầu: {last_comment_count}")

    while click_count < 50:
        try:
            # Cập nhật XPath để tìm chính xác nút "XXX đã trả lời Y phản hồi"
            reply_buttons = browser.find_elements(By.XPATH, "//div[@role='button' and contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'đã trả lời') and contains(., 'phản hồi')]")
            if not reply_buttons:
                print("Không còn nút 'XXX đã trả lời Y phản hồi'.")
                break
            clicked = False
            for button in reply_buttons:
                try:
                    button_text = button.text.strip()
                    browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                    if button.is_displayed() and button.is_enabled():
                        wait.until(EC.element_to_be_clickable(button)).click()
                        print(f"Đã nhấp nút: '{button_text}'")
                        time.sleep(1)  # Tăng thời gian chờ để phản hồi tải
                        click_count += 1
                        clicked = True
                        current_comment_count = count_comments()
                        print(f"Số bình luận hiện tại: {current_comment_count}")
                        if current_comment_count <= last_comment_count:
                            no_new_comments_count += 1
                            print(f"Không có bình luận mới (lần {no_new_comments_count}/3).")
                            if no_new_comments_count >= 3:
                                print("Không có bình luận mới sau 3 lần, dừng nhấp.")
                                return click_count > 0
                        else:
                            no_new_comments_count = 0
                        last_comment_count = current_comment_count
                        scroll_to_bottom()
                    else:
                        print(f"Nút không hợp lệ: '{button_text}', bỏ qua.")
                except Exception as e:
                    print(f"Không thể nhấp nút: '{button_text}', lỗi: {str(e)}, bỏ qua.")
                    continue
            if not clicked:
                print("Không nhấp được nút nào, dừng nhấp.")
                break
            scroll_to_bottom()
            time.sleep(0.5)
        except Exception as e:
            print(f"Không tìm thấy nút 'XXX đã trả lời Y phản hồi', lỗi: {str(e)}, dừng nhấp.")
            break
    if click_count >= 50:
        print("Đạt giới hạn 50 lần nhấp, dừng nhấp.")
    return click_count > 0

# Hàm nhấp vào nút "Xem thêm bình luận"
def click_view_more_comments():
    print("Đang nhấp vào các nút 'Xem thêm bình luận'...")
    max_attempts = 5
    click_count = 0
    no_new_comments_count = 0
    last_comment_count = count_comments()
    print(f"Số bình luận ban đầu: {last_comment_count}")

    while click_count < 50:
        try:
            view_more_buttons = browser.find_elements(By.XPATH, "//div[@role='button' and contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'xem thêm bình luận')]")
            if not view_more_buttons:
                print("Không còn nút 'Xem thêm bình luận'.")
                break
            clicked = False
            for button in view_more_buttons:
                try:
                    button_text = button.text.strip()
                    browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                    if button.is_displayed() and button.is_enabled():
                        wait.until(EC.element_to_be_clickable(button)).click()
                        print(f"Đã nhấp nút: '{button_text}'")
                        time.sleep(0.5)
                        click_count += 1
                        clicked = True
                        current_comment_count = count_comments()
                        print(f"Số bình luận hiện tại: {current_comment_count}")
                        if current_comment_count <= last_comment_count:
                            no_new_comments_count += 1
                            print(f"Không có bình luận mới (lần {no_new_comments_count}/3).")
                            if no_new_comments_count >= 3:
                                print("Không có bình luận mới sau 3 lần, dừng nhấp.")
                                return click_count > 0
                        else:
                            no_new_comments_count = 0
                        last_comment_count = current_comment_count
                        scroll_to_bottom()
                    else:
                        print(f"Nút không hợp lệ: '{button_text}', bỏ qua.")
                except Exception as e:
                    print(f"Không thể nhấp nút: '{button_text}', lỗi: {str(e)}, bỏ qua.")
                    continue
            if not clicked:
                print("Không nhấp được nút nào, dừng nhấp.")
                break
            time.sleep(0.5)
        except Exception as e:
            print(f"Không tìm thấy nút 'Xem thêm bình luận', lỗi: {str(e)}, dừng nhấp.")
            break
    if click_count >= 50:
        print("Đạt giới hạn 50 lần nhấp, dừng nhấp.")
    return click_count > 0

# Hàm nhấp vào "Phù hợp nhất" và chọn "Tất cả bình luận"
def select_all_comments():
    print("Đang chọn 'Tất cả bình luận'...")
    try:
        filter_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@role='button' and contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'phù hợp nhất')]")
        ))
        browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", filter_button)
        filter_button.click()
        time.sleep(0.5)

        all_comments_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//span[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'tất cả bình luận')]")
        ))
        browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", all_comments_button)
        all_comments_button.click()
        time.sleep(0.5)
        print("Đã chọn 'Tất cả bình luận'.")
        return True
    except Exception as e:
        print(f"Không thể chọn 'Tất cả bình luận': {str(e)}")
        return False

# Hàm xử lý một bài viết
def process_post(post_url, post_time, post_content, post_index):
    print(f"Đang mở bài viết: {post_url}")
    browser.get(post_url)
    time.sleep(3)

    select_all_comments()
    scroll_to_bottom()
    click_view_more_comments()
    click_see_replies()
    click_see_more()

    time.sleep(0.5)
    browser.execute_script("window.scrollTo(0, 0);")
    time.sleep(0.5)

    output_dir = "HTTSV2"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html.parser")

    html_filename = os.path.join(output_dir, f"facebook_post_{post_index}.html")
    with open(html_filename, "w", encoding="utf-8") as file:
        file.write(soup.prettify())

    author_div = soup.find("div", class_="html-div xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1iyjqo2")
    post_author_id = "Không rõ ID"
    post_author_name = "Không rõ"

    if author_div:
        author_link = author_div.find("a")
        if author_link and author_link.has_attr("href"):
            match = re.search(r"user/(\d+)/", author_link["href"])
            if match:
                post_author_id = match.group(1)
            post_author_name = author_link.get_text(strip=True)

    if post_author_id == "Không rõ ID":
        post_author_name = "@Người tham gia ẩn danh"

    like_count_span = soup.find("span", class_="xt0b8zv x1e558r4")
    like_count = like_count_span.get_text(strip=True) if like_count_span else "0"

    comment_share_spans = soup.find_all("span", class_="html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs xkrqix3 x1sur9pj")
    comment_count = comment_share_spans[0].get_text(strip=True) if len(comment_share_spans) > 0 else "0"
    share_count = comment_share_spans[1].get_text(strip=True) if len(comment_share_spans) > 1 else "0"

    comment_clusters = soup.find_all("div", class_="x16hk5td x12rz0ws")
    comments_json = {}

    def extract_comment_id(div):
        comment_link = div.find("a", class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xkrqix3 x1sur9pj xi81zsa x1s688f")
        if comment_link and comment_link.has_attr("href"):
            href = comment_link["href"]
            match_reply = re.search(r"reply_comment_id=(\d+)", href)
            match_comment = re.search(r"comment_id=(\d+)", href)
            if match_reply:
                return match_reply.group(1)
            elif match_comment:
                return match_comment.group(1)
        return "Không rõ ID"

    for index, div in enumerate(comment_clusters, start=1):
        key = f"comment{index}"

        activity_div = div.find("div", class_="x1n2onr6 x1ye3gou x1iorvi4 x78zum5 x1q0g3np x1a2a7pz")
        activity_text = activity_div.get("aria-label", "Không rõ hoạt động") if activity_div else "Không rõ hoạt động"

        user_link = div.find("a", class_="x1i10hfl")
        user_id = "Không rõ ID"
        if user_link and user_link.has_attr("href"):
            match = re.search(r"user/(\d+)/", user_link["href"])
            if match:
                user_id = match.group(1)

        user_name_tag = div.find("span", class_="x193iq5w")
        user_name = user_name_tag.get_text(strip=True) if user_name_tag else "Không rõ"

        if user_name.startswith("Người tham gia ẩn danh"):
            user_id = "Không rõ"

        content_div = div.find("div", class_="x1lliihq xjkvuk6 x1iorvi4")
        comment_text = content_div.get_text(" ", strip=True) if content_div else "Không có nội dung"

        mentioned_users = []
        mentioned_user_ids = []
        if content_div:
            for name_span in content_div.find_all("span", attrs={"aria-describedby": True}):
                tagged_name = name_span.get_text(strip=True)
                if tagged_name and tagged_name != user_name:
                    mentioned_users.append(tagged_name)
                    comment_text = comment_text.replace(tagged_name, f"@{tagged_name}")

            mentioned_links = content_div.find_all("a", class_="x1i10hfl")
            for link in mentioned_links:
                if link.has_attr("href"):
                    match = re.search(r"user/(\d+)/", link["href"])
                    if match:
                        mentioned_user_ids.append(match.group(1))

        comments_json[key] = {
            "activity": activity_text,
            "user-id": user_id,
            "user_name": user_name,
            "comment-id": extract_comment_id(div),
            "content-comment": comment_text,
            "mentioned-users": mentioned_users,
            "mentioned-user-ids": mentioned_user_ids,
            "replies": {}
        }

        reply_divs = div.find_all("div", class_="x1n2onr6 x1e558r4 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz")
        subkey_dict = comments_json[key]["replies"]
        prev_subcomment = None
        subsubcomment_counter = {}

        for reply_index, reply_div in enumerate(reply_divs, start=1):
            subcomment = f"subcomment{reply_index}"
            reply_activity_text = reply_div.get("aria-label", "Không rõ hoạt động")

            reply_user_link = reply_div.find("a", class_="x1i10hfl")
            reply_user_id = "Không rõ ID"
            if reply_user_link and reply_user_link.has_attr("href"):
                match = re.search(r"user/(\d+)/", reply_user_link["href"])
                if match:
                    reply_user_id = match.group(1)

            reply_user_name_tag = reply_div.find("span", class_="x193iq5w")
            reply_user_name = reply_user_name_tag.get_text(strip=True) if reply_user_name_tag else "Không rõ"

            if reply_user_name.startswith("Người tham gia ẩn danh"):
                reply_user_id = "Không rõ"

            reply_content_div = reply_div.find("div", class_="x1lliihq xjkvuk6 x1iorvi4")
            reply_text = reply_content_div.get_text(" ", strip=True) if reply_content_div else "Không có nội dung"

            reply_mentioned_users = []
            reply_mentioned_user_ids = []
            if reply_content_div:
                for name_span in reply_content_div.find_all("span", attrs={"aria-describedby": True}):
                    tagged_name = name_span.get_text(strip=True)
                    if tagged_name:
                        reply_mentioned_users.append(tagged_name)
                        reply_text = reply_text.replace(tagged_name, f"@{tagged_name}")
                for link in reply_content_div.find_all("a", class_="x1i10hfl"):
                    if link.has_attr("href"):
                        match = re.search(r"user/(\d+)/", link["href"])
                        if match:
                            reply_mentioned_user_ids.append(match.group(1))

            reply_data = {
                "activity": reply_activity_text,
                "user-id": reply_user_id,
                "user_name": reply_user_name,
                "comment-id": extract_comment_id(reply_div),
                "content-comment": reply_text,
                "mentioned-users": reply_mentioned_users,
                "mentioned-user-ids": reply_mentioned_user_ids,
                "replies": {}
            }

            if prev_subcomment and "đáp lại phản hồi của" in reply_activity_text:
                if prev_subcomment not in subsubcomment_counter:
                    subsubcomment_counter[prev_subcomment] = 1
                else:
                    subsubcomment_counter[prev_subcomment] += 1

                subsubcomment_key = f"subsubcomment{subsubcomment_counter[prev_subcomment]}"
                subkey_dict[prev_subcomment]["replies"][subsubcomment_key] = reply_data
            else:
                subkey_dict[subcomment] = reply_data
                prev_subcomment = subcomment
                subsubcomment_counter[subcomment] = 0

    output_data = {
        "post_url": post_url,
        "time": post_time,
        "post_author_id": post_author_id,
        "post_author_name": post_author_name,
        "post_content": post_content.replace("text:", "").strip(),
        "likeCount": like_count,
        "commentCount": comment_count,
        "shareCount": share_count,
        "comments": comments_json
    }

    json_filename = os.path.join(output_dir, f"facebook_post_data_{post_index}.json")
    with open(json_filename, "w", encoding="utf-8") as file:
        json.dump(output_data, file, ensure_ascii=False, indent=4)

    if os.path.exists(html_filename):
        os.remove(html_filename)

    print(f"Đã xử lý bài viết {post_index}: {post_url} - JSON lưu tại {json_filename}")

# Main execution
def main():
    login_facebook()
    url_file = "links_HTTSV2.txt"
    post_data = []
    with open(url_file, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                parts = line.split("\t")
                if len(parts) != 2:
                    print(f"Dòng không đúng định dạng: {line}")
                    continue
                url = parts[0]
                time_and_text = parts[1].split(" ", 1)
                if len(time_and_text) != 2:
                    post_time = time_and_text[0]
                    post_content = "text: Không có nội dung"
                else:
                    post_time, post_content = time_and_text
                post_data.append((url, post_time, post_content))

    for index, (post_url, post_time, post_content) in enumerate(post_data, start=1):
        try:
            print(f"Bắt đầu xử lý bài viết {index}/{len(post_data)}")
            process_post(post_url, post_time, post_content, index)
        except Exception as e:
            print(f"Lỗi khi xử lý bài viết {post_url}: {str(e)}")
            with open("error_log.txt", "a", encoding="utf-8") as log_file:
                log_file.write(f"Bài viết {post_url}: {str(e)}\n")
        time.sleep(5)

    browser.quit()

if __name__ == "__main__":
    main()